In [2]:
import time
import os
import json
import pickle
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50, VGG19
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import losses

from matplotlib.pyplot import imread, imshow

import h5py
import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime
import nibabel as nib
import re
from collections import Counter


%matplotlib inline
%load_ext autoreload
%autoreload 1

SEED=1
random.seed(SEED)
np.random.seed(SEED)
#tf.set_random_seed(SEED)
tf.random.set_seed

K.clear_session()



In [61]:

# Get fmri ROI data
import h5py
tr_list = []
for roi in ['RHPPA', 'LHPPA', 'RHRSC', 'LHRSC', 'RHOPA', 'LHOPA']:
    for TR in ['TR3']:
        with h5py.File('/home/ubuntu/boldroi/ROIs/CSI1/h5/CSI1_ROIs_%s.h5' % TR, 'r') as f:
            print(list(f.keys()))
            x_roi = list(f[roi])
            x_r = np.stack(x_roi, axis=0)
            tr_list.append(x_r)
            #mean_x = np.mean(x_r, axis=0)
            #std_x = np.std(x_r, axis=0, ddof=1)
            #xt = (x_r - mean_x) / std_x
            #tr_list.append(xt)
            #xt_pad = np.pad(xt, ((0, 0), (0, ax_length-xt.shape[1])), mode='constant', constant_values=0)
            #tr_list.append(xt_pad)

    #x_stack = np.dstack(tr_list)
    #print(x_stack.shape)
    #xsa = np.swapaxes(x_stack,1,2)
    #roi_list.append(xsa)

#x_all = np.concatenate(roi_list)
#x_all =np.squeeze(x_all)
#print(x_all.shape)
for roi in tr_list:
    print(roi.shape)


['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
['LHEarlyVis', 'LHLOC', 'LHOPA', 'LHPPA', 'LHRSC', 'RHEarlyVis', 'RHLOC', 'RHOPA', 'RHPPA', 'RHRSC']
(5254, 200)
(5254, 131)
(5254, 143)
(5254, 86)
(5254, 187)
(5254, 101)


In [29]:
# load stimuli list
with open('/home/ubuntu/boldroi/ROIs/stim_lists/CSI01_stim_lists.txt', 'r') as f:
    stimuli_list = f.read().splitlines()

classes = {'ImageNet': 0, 'COCO': 1, 'Scene': 2}
stimulusDirPath = '/home/ubuntu/fmriNet/fmriNet/images/BOLD5000_Stimuli/Scene_Stimuli/Presented_Stimuli'
stimuli_file_list = []
label_list = []
for imageFileName in stimuli_list:
    for (currDir, _, fileList) in os.walk(stimulusDirPath):
        currBaseDir = os.path.basename(currDir)
        for filename in fileList:
            if filename in imageFileName:
                stimuli_file_list.append(os.path.join(currDir, filename))
                label_list.append(classes.get(currBaseDir))
                break

labels = np.asarray(label_list)
print(labels.shape)
labels_all = utils.to_categorical(labels)
# CSI01 (5254,)
# CSI02 (5254,)
# CSI03 (5254,)
# CSI03 (3108,)


(5254,)


In [11]:

# load stimuli activations
with open('imageActivation/CSI01_renet_avg_pool_activations.npy', 'rb') as f:
    y_all_orig = pickle.load(f)

print(y_all_orig.shape)

(5254, 2048)


In [9]:
# borrowed from bdpy https://github.com/KamitaniLab/bdpy 
from numpy.matlib import repmat
def corrcoef(x, y, var='row'):
    """
    Returns correlation coefficients between `x` and `y`

    Parameters
    ----------
    x, y : array_like
        Matrix or vector
    var : str, 'row' or 'col'
        Specifying whether rows (default) or columns represent variables

    Returns
    -------
    r
        Correlation coefficients
    """

    # Convert vectors to arrays
    if x.ndim == 1:
        x = np.array([x])

    if y.ndim == 1:
        y = np.array([y])

    # Normalize x and y to row-var format
    if var == 'row':
        # 'rowvar=1' in np.corrcoef

        # Vertical vector --> horizontal
        if x.shape[1] == 1:
            x = x.T
        if y.shape[1] == 1:
            y = y.T
    elif var == 'col':
        # 'rowvar=0' in np.corrcoef

        # Horizontal vector --> vertical
        if x.shape[0] == 1:
            x = x.T
        if y.shape[0] == 1:
            y = y.T

        # Convert to rowvar=1
        x = x.T
        y = y.T
    else:
        raise ValueError('Unknown var parameter specified')

    # Match size of x and y
    if x.shape[0] == 1 and y.shape[0] != 1:
        x = repmat(x, y.shape[0], 1)
        
    elif x.shape[0] != 1 and y.shape[0] == 1:
        y = repmat(y, x.shape[0], 1)

    # Check size of normalized x and y
    if x.shape != y.shape:
        raise TypeError('Input matrixes size mismatch')

    # Get num variables
    nvar = x.shape[0]

    # Get correlation
    rmat = np.corrcoef(x, y, rowvar=1)
    r = np.diag(rmat[:nvar, nvar:])

    return r

# borrowed from bdpy https://github.com/KamitaniLab/bdpy 
def select_top(data, value, num, axis=0, verbose=True):
    """
    Select top `num` features of `value` from `data`

    Parameters
    ----------
    data : array
       Data matrix
    value : array_like
       Vector of values
    num : int
       Number of selected features

    Returns
    -------
    selected_data : array
        Selected data matrix
    selected_index : array
        Index of selected data
    """

    if verbose:
        print_start_msg()

    num_elem = data.shape[axis]

    sorted_index = np.argsort(value)[::-1]

    rank = np.zeros(num_elem, dtype=np.int)
    rank[sorted_index] = np.array(range(0, num_elem))

    selected_index_bool = rank < num

    if axis == 0:
        selected_data = data[selected_index_bool, :]
        selected_index = np.array(range(0, num_elem), dtype=np.int)[selected_index_bool]
    elif axis == 1:
        selected_data = data[:, selected_index_bool]
        selected_index = np.array(range(0, num_elem), dtype=np.int)[selected_index_bool]
    else:
        raise ValueError('Invalid axis')

    if verbose:
        print_finish_msg()

    return selected_data, selected_index

# borrowed from bdpy https://github.com/KamitaniLab/bdpy 
def add_bias(x, axis=0):
    """
    Add bias terms to x

    Parameters
    ----------
    x : array_like
        Data matrix
    axis : 0 or 1, optional
        Axis in which bias terms are added (default: 0)

    Returns
    -------
    y : array_like
        Data matrix with bias terms
    """

    if axis == 0:
        vlen = x.shape[1]
        y = np.concatenate((x, np.array([np.ones(vlen)])), axis=0)
    elif axis == 1:
        vlen = x.shape[0]
        y = np.concatenate((x, np.array([np.ones(vlen)]).T), axis=1)
    else:
        raise ValueError('axis should be either 0 or 1')
    
    return y

In [ ]:
import collections
# Sample voxels most correlated with average categoris
# loop for each features in each of 3 categories, resample x_train
num_voxels = 50

x_all_list = []
for roi_voxels in tr_list:
    voxel_counter = collections.Counter()
    #for feature in index_list:
    for feature in range(0, y_all_orig.shape[1]):
        print("feature: %s" % (feature))
        y_train_unit = y_all_orig[:, feature]
        norm_mean_y = np.mean(y_train_unit, axis=0)
        std_y = np.std(y_train_unit, axis=0, ddof=1)
        norm_scale_y = 1 if std_y == 0 else std_y
        y_train_unit = (y_train_unit - norm_mean_y) / norm_scale_y
        # Voxel selection
        corr = corrcoef(y_train_unit, roi_voxels, var='col')
        x_train_unit, selected_index = select_top(roi_voxels, np.abs(corr), num_voxels, axis=1, verbose=False)
        voxel_counter.update(selected_index)
    
    voxel_list = [voxel for voxel, count in voxel_counter.most_common(num_voxels)]
    x_all_list.append(roi_voxels[:, voxel_list])

print(x_all_list[0].shape)
print(x_all_list[1].shape)

feature: 0
feature: 1
feature: 2
feature: 3
feature: 4
feature: 5
feature: 6
feature: 7
feature: 8
feature: 9
feature: 10
feature: 11
feature: 12
feature: 13
feature: 14
feature: 15
feature: 16
feature: 17
feature: 18
feature: 19
feature: 20
feature: 21
feature: 22
feature: 23
feature: 24
feature: 25
feature: 26
feature: 27
feature: 28
feature: 29
feature: 30
feature: 31
feature: 32
feature: 33
feature: 34
feature: 35
feature: 36
feature: 37
feature: 38
feature: 39
feature: 40
feature: 41
feature: 42
feature: 43
feature: 44
feature: 45
feature: 46
feature: 47
feature: 48
feature: 49
feature: 50
feature: 51
feature: 52
feature: 53
feature: 54
feature: 55
feature: 56
feature: 57
feature: 58
feature: 59
feature: 60
feature: 61
feature: 62
feature: 63
feature: 64
feature: 65
feature: 66
feature: 67
feature: 68
feature: 69
feature: 70
feature: 71
feature: 72
feature: 73
feature: 74
feature: 75
feature: 76
feature: 77
feature: 78
feature: 79
feature: 80
feature: 81
feature: 82
feature: 83
fe

feature: 641
feature: 642
feature: 643
feature: 644
feature: 645
feature: 646
feature: 647
feature: 648
feature: 649
feature: 650
feature: 651
feature: 652
feature: 653
feature: 654
feature: 655
feature: 656
feature: 657
feature: 658
feature: 659
feature: 660
feature: 661
feature: 662
feature: 663
feature: 664
feature: 665
feature: 666
feature: 667
feature: 668
feature: 669
feature: 670
feature: 671
feature: 672
feature: 673
feature: 674
feature: 675
feature: 676
feature: 677
feature: 678
feature: 679
feature: 680
feature: 681
feature: 682
feature: 683
feature: 684
feature: 685
feature: 686
feature: 687
feature: 688
feature: 689
feature: 690
feature: 691
feature: 692
feature: 693
feature: 694
feature: 695
feature: 696
feature: 697
feature: 698
feature: 699
feature: 700
feature: 701
feature: 702
feature: 703
feature: 704
feature: 705
feature: 706
feature: 707
feature: 708
feature: 709
feature: 710
feature: 711
feature: 712
feature: 713
feature: 714
feature: 715
feature: 716
feature: 717

feature: 1253
feature: 1254
feature: 1255
feature: 1256
feature: 1257
feature: 1258
feature: 1259
feature: 1260
feature: 1261
feature: 1262
feature: 1263
feature: 1264
feature: 1265
feature: 1266
feature: 1267
feature: 1268
feature: 1269
feature: 1270
feature: 1271
feature: 1272
feature: 1273
feature: 1274
feature: 1275
feature: 1276
feature: 1277
feature: 1278
feature: 1279
feature: 1280
feature: 1281
feature: 1282
feature: 1283
feature: 1284
feature: 1285
feature: 1286
feature: 1287
feature: 1288
feature: 1289
feature: 1290
feature: 1291
feature: 1292
feature: 1293
feature: 1294
feature: 1295
feature: 1296
feature: 1297
feature: 1298
feature: 1299
feature: 1300
feature: 1301
feature: 1302
feature: 1303
feature: 1304
feature: 1305
feature: 1306
feature: 1307
feature: 1308
feature: 1309
feature: 1310
feature: 1311
feature: 1312
feature: 1313
feature: 1314
feature: 1315
feature: 1316
feature: 1317
feature: 1318
feature: 1319
feature: 1320
feature: 1321
feature: 1322
feature: 1323
featur

feature: 1840
feature: 1841
feature: 1842
feature: 1843
feature: 1844
feature: 1845
feature: 1846
feature: 1847
feature: 1848
feature: 1849
feature: 1850
feature: 1851
feature: 1852
feature: 1853
feature: 1854
feature: 1855
feature: 1856
feature: 1857
feature: 1858
feature: 1859
feature: 1860
feature: 1861
feature: 1862
feature: 1863
feature: 1864
feature: 1865
feature: 1866
feature: 1867
feature: 1868
feature: 1869
feature: 1870
feature: 1871
feature: 1872
feature: 1873
feature: 1874
feature: 1875
feature: 1876
feature: 1877
feature: 1878
feature: 1879
feature: 1880
feature: 1881
feature: 1882
feature: 1883
feature: 1884
feature: 1885
feature: 1886
feature: 1887
feature: 1888
feature: 1889
feature: 1890
feature: 1891
feature: 1892
feature: 1893
feature: 1894
feature: 1895
feature: 1896
feature: 1897
feature: 1898
feature: 1899
feature: 1900
feature: 1901
feature: 1902
feature: 1903
feature: 1904
feature: 1905
feature: 1906
feature: 1907
feature: 1908
feature: 1909
feature: 1910
featur

feature: 422
feature: 423
feature: 424
feature: 425
feature: 426
feature: 427
feature: 428
feature: 429
feature: 430
feature: 431
feature: 432
feature: 433
feature: 434
feature: 435
feature: 436
feature: 437
feature: 438
feature: 439
feature: 440
feature: 441
feature: 442
feature: 443
feature: 444
feature: 445
feature: 446
feature: 447
feature: 448
feature: 449
feature: 450
feature: 451
feature: 452
feature: 453
feature: 454
feature: 455
feature: 456
feature: 457
feature: 458
feature: 459
feature: 460
feature: 461
feature: 462
feature: 463
feature: 464
feature: 465
feature: 466
feature: 467
feature: 468
feature: 469
feature: 470
feature: 471
feature: 472
feature: 473
feature: 474
feature: 475
feature: 476
feature: 477
feature: 478
feature: 479
feature: 480
feature: 481
feature: 482
feature: 483
feature: 484
feature: 485
feature: 486
feature: 487
feature: 488
feature: 489
feature: 490
feature: 491
feature: 492
feature: 493
feature: 494
feature: 495
feature: 496
feature: 497
feature: 498

...

feature: 1051
feature: 1052
feature: 1053
feature: 1054
feature: 1055
feature: 1056
feature: 1057
feature: 1058
feature: 1059
feature: 1060
feature: 1061
feature: 1062
feature: 1063
feature: 1064
feature: 1065
feature: 1066
feature: 1067
feature: 1068
feature: 1069
feature: 1070
feature: 1071
feature: 1072
feature: 1073
feature: 1074
feature: 1075
feature: 1076
feature: 1077
feature: 1078
feature: 1079
feature: 1080
feature: 1081
feature: 1082
feature: 1083
feature: 1084
feature: 1085
feature: 1086
feature: 1087
feature: 1088
feature: 1089
feature: 1090
feature: 1091
feature: 1092
feature: 1093
feature: 1094
feature: 1095
feature: 1096
feature: 1097
feature: 1098
feature: 1099
feature: 1100
feature: 1101
feature: 1102
feature: 1103
feature: 1104
feature: 1105
feature: 1106
feature: 1107
feature: 1108
feature: 1109
feature: 1110
feature: 1111
feature: 1112
feature: 1113
feature: 1114
feature: 1115
feature: 1116
feature: 1117
feature: 1118
feature: 1119
feature: 1120
feature: 1121
featur

feature: 1640
feature: 1641
feature: 1642
feature: 1643
feature: 1644
feature: 1645
feature: 1646
feature: 1647
feature: 1648
feature: 1649
feature: 1650
feature: 1651
feature: 1652
feature: 1653
feature: 1654
feature: 1655
feature: 1656
feature: 1657
feature: 1658
feature: 1659
feature: 1660
feature: 1661
feature: 1662
feature: 1663
feature: 1664
feature: 1665
feature: 1666
feature: 1667
feature: 1668
feature: 1669
feature: 1670
feature: 1671
feature: 1672
feature: 1673
feature: 1674
feature: 1675
feature: 1676
feature: 1677
feature: 1678
feature: 1679
feature: 1680
feature: 1681
feature: 1682
feature: 1683
feature: 1684
feature: 1685
feature: 1686
feature: 1687
feature: 1688
feature: 1689
feature: 1690
feature: 1691
feature: 1692
feature: 1693
feature: 1694
feature: 1695
feature: 1696
feature: 1697
feature: 1698
feature: 1699
feature: 1700
feature: 1701
feature: 1702
feature: 1703
feature: 1704
feature: 1705
feature: 1706
feature: 1707
feature: 1708
feature: 1709
feature: 1710
featur

feature: 212
feature: 213
feature: 214
feature: 215
feature: 216
feature: 217
feature: 218
feature: 219
feature: 220
feature: 221
feature: 222
feature: 223
feature: 224
feature: 225
feature: 226
feature: 227
feature: 228
feature: 229
feature: 230
feature: 231
feature: 232
feature: 233
feature: 234
feature: 235
feature: 236
feature: 237
feature: 238
feature: 239
feature: 240
feature: 241
feature: 242
feature: 243
feature: 244
feature: 245
feature: 246
feature: 247
feature: 248
feature: 249
feature: 250
feature: 251
feature: 252
feature: 253
feature: 254
feature: 255
feature: 256
feature: 257
feature: 258
feature: 259
feature: 260
feature: 261
feature: 262
feature: 263
feature: 264
feature: 265
feature: 266
feature: 267
feature: 268
feature: 269
feature: 270
feature: 271
feature: 272
feature: 273
feature: 274
feature: 275
feature: 276
feature: 277
feature: 278
feature: 279
feature: 280
feature: 281
feature: 282
feature: 283
feature: 284
feature: 285
feature: 286
feature: 287
feature: 288

feature: 854
feature: 855
feature: 856
feature: 857
feature: 858
feature: 859
feature: 860
feature: 861
feature: 862
feature: 863
feature: 864
feature: 865
feature: 866
feature: 867
feature: 868
feature: 869
feature: 870
feature: 871
feature: 872
feature: 873
feature: 874
feature: 875
feature: 876
feature: 877
feature: 878
feature: 879
feature: 880
feature: 881
feature: 882
feature: 883
feature: 884
feature: 885
feature: 886
feature: 887
feature: 888
feature: 889
feature: 890
feature: 891
feature: 892
feature: 893
feature: 894
feature: 895
feature: 896
feature: 897
feature: 898
feature: 899
feature: 900
feature: 901
feature: 902
feature: 903
feature: 904
feature: 905
feature: 906
feature: 907
feature: 908
feature: 909
feature: 910
feature: 911
feature: 912
feature: 913
feature: 914
feature: 915
feature: 916
feature: 917
feature: 918
feature: 919
feature: 920
feature: 921
feature: 922
feature: 923
feature: 924
feature: 925
feature: 926
feature: 927
feature: 928
feature: 929
feature: 930

feature: 1454
feature: 1455
feature: 1456
feature: 1457
feature: 1458
feature: 1459
feature: 1460
feature: 1461
feature: 1462
feature: 1463
feature: 1464
feature: 1465
feature: 1466
feature: 1467
feature: 1468
feature: 1469
feature: 1470
feature: 1471
feature: 1472
feature: 1473
feature: 1474
feature: 1475
feature: 1476
feature: 1477
feature: 1478
feature: 1479
feature: 1480
feature: 1481
feature: 1482
feature: 1483
feature: 1484
feature: 1485
feature: 1486
feature: 1487
feature: 1488
feature: 1489
feature: 1490
feature: 1491
feature: 1492
feature: 1493
feature: 1494
feature: 1495
feature: 1496
feature: 1497
feature: 1498
feature: 1499
feature: 1500
feature: 1501
feature: 1502
feature: 1503
feature: 1504
feature: 1505
feature: 1506
feature: 1507
feature: 1508
feature: 1509
feature: 1510
feature: 1511
feature: 1512
feature: 1513
feature: 1514
feature: 1515
feature: 1516
feature: 1517
feature: 1518
feature: 1519
feature: 1520
feature: 1521
feature: 1522
feature: 1523
feature: 1524
featur

In [63]:
x_all = np.hstack(x_all_list)
print(x_all.shape)

(5254, 480)


In [163]:
#from numpy.linalg import norm
## for each fmri sample, multiply roi by cat average and take the vector with highest cosine similarity to train on
#for sample in range(0, 10):
#    print("--------")
#    print(labels[sample])
#    x_curr = x_all[sample, :]
#    cat_ave = average_activations[0][0:200]
#    print("cosine similarity: %s" % np.divide(np.dot(x_curr, cat_ave),(norm(x_curr)*norm(cat_ave))))
#    cat_ave = average_activations[1][0:200]
#    print("cosine similarity: %s" % np.divide(np.dot(x_curr, cat_ave),(norm(x_curr)*norm(cat_ave))))
#    cat_ave = average_activations[2][0:200]
#    print("cosine similarity: %s" % np.divide(np.dot(x_curr, cat_ave),(norm(x_curr)*norm(cat_ave))))
    
    


In [77]:
# Split data into train/test set
from sklearn.utils import shuffle
num_samples = x_all.shape[0]
x_shuffle, y_shuffle = shuffle(x_all, labels_all, random_state=0)
div = int(num_samples * 0.9)
x_train = x_shuffle[0:div, :]
y_train = y_shuffle[0:div]
x_test = x_shuffle[div:, :]
y_test = y_shuffle[div:]

#num_samples = x_all.shape[0]
##num_voxels = 100  # maximum number of voxels to use for training
#div = int(num_samples * 0.6)
#print("Division index: %s" % div)
##x_shuffle, y_shuffle = shuffle(x_all, y_all, random_state=0)
#x_train = x_all[0:div, :]
##y_train = y_all[0:div, :]
#y_train = labels_all[0:div]
#
#x_test = x_all[div:, :]
##y_test = y_all[div:, :]
#y_test = labels_all[div:]
print("x train shape: %s" % str(x_train.shape))
print("y train shape: %s" % str(y_train.shape))
print("x test shape: %s" % str(x_test.shape))
print("y test shape: %s" % str(y_test.shape))


x train shape: (4728, 480)
y train shape: (4728, 3)
x test shape: (526, 480)
y test shape: (526, 3)


In [78]:

X_input = Input(shape=(x_train.shape[1]))
X = Dense(32, activation='relu')(X_input)
X = Dropout(0.2)(X)
X = Dense(4, activation='relu')(X)
predictions = Dense(3, activation='softmax')(X)

#X_input = Input(shape=(x_train.shape[1]))
#X = Dense(4, activation='relu')(X_input)
#X = Dropout(0.2)(X)
#X = Dense(32, activation='relu')(X)
#X = Dropout(0.2)(X)
#X = Dense(32, activation='relu')(X)
#X = Dropout(0.2)(X)
#X = Dense(32, activation='relu')(X)
#X = Dropout(0.2)(X)
#predictions = Dense(3, activation='softmax')(X)

classifier_model = Model(inputs=X_input, outputs=predictions)
classifier_model.summary()


i=0
callbacks = [TensorBoard(log_dir=f'./log/{i}')]

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 480)]             0         
_________________________________________________________________
dense_47 (Dense)             (None, 32)                15392     
_________________________________________________________________
dropout_23 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 4)                 132       
_________________________________________________________________
dense_49 (Dense)             (None, 3)                 15        
Total params: 15,539
Trainable params: 15,539
Non-trainable params: 0
_________________________________________________________________


In [79]:


from tensorflow.keras import optimizers
EPOCHS=500
callbacks = [TensorBoard(log_dir=f'./log/{i}')]
#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
#classifier_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
#classifier_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
classifier_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier_model.fit(x=x_train, y=y_train, shuffle=True, epochs=EPOCHS, callbacks=callbacks, validation_data=(x_test, y_test))
#classifier_model.fit(x=x_train, y=y_train, epochs=EPOCHS, callbacks=callbacks, validation_data=(x_test, y_test))



Train on 4728 samples, validate on 526 samples
Epoch 1/500
4728/4728 [==============================] - 1s 161us/sample - loss: 1.0563 - accuracy: 0.4281 - val_loss: 0.9947 - val_accuracy: 0.5038
Epoch 2/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.9715 - accuracy: 0.5008 - val_loss: 0.9353 - val_accuracy: 0.5722
Epoch 3/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.9281 - accuracy: 0.5338 - val_loss: 0.9111 - val_accuracy: 0.5798
Epoch 4/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.8970 - accuracy: 0.5615 - val_loss: 0.8962 - val_accuracy: 0.5894
Epoch 5/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.8762 - accuracy: 0.5757 - val_loss: 0.8877 - val_accuracy: 0.5779
Epoch 6/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.8586 - accuracy: 0.5924 - val_loss: 0.8807 - val_accuracy: 0.5951
Epoch 7/500
4728/4728 [==============================] - 0s 56us

Epoch 56/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.3703 - accuracy: 0.8625 - val_loss: 1.3466 - val_accuracy: 0.5133
Epoch 57/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.3657 - accuracy: 0.8676 - val_loss: 1.3790 - val_accuracy: 0.5171
Epoch 58/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.3501 - accuracy: 0.8773 - val_loss: 1.3883 - val_accuracy: 0.5095
Epoch 59/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.3476 - accuracy: 0.8758 - val_loss: 1.4222 - val_accuracy: 0.5038
Epoch 60/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.3416 - accuracy: 0.8763 - val_loss: 1.4254 - val_accuracy: 0.5133
Epoch 61/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.3463 - accuracy: 0.8739 - val_loss: 1.4326 - val_accuracy: 0.5209
Epoch 62/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.3353 - accuracy: 0.8824

Epoch 111/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.1527 - accuracy: 0.9490 - val_loss: 2.3315 - val_accuracy: 0.4962
Epoch 112/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.1389 - accuracy: 0.9516 - val_loss: 2.3487 - val_accuracy: 0.5152
Epoch 113/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.1453 - accuracy: 0.9543 - val_loss: 2.3569 - val_accuracy: 0.5095
Epoch 114/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.1425 - accuracy: 0.9543 - val_loss: 2.3708 - val_accuracy: 0.5038
Epoch 115/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.1349 - accuracy: 0.9543 - val_loss: 2.3766 - val_accuracy: 0.5057
Epoch 116/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.1332 - accuracy: 0.9552 - val_loss: 2.4183 - val_accuracy: 0.4924
Epoch 117/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.1205 - accuracy:

4728/4728 [==============================] - 0s 56us/sample - loss: 0.0888 - accuracy: 0.9698 - val_loss: 3.1084 - val_accuracy: 0.4924
Epoch 166/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.0845 - accuracy: 0.9708 - val_loss: 3.1828 - val_accuracy: 0.4905
Epoch 167/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.0926 - accuracy: 0.9681 - val_loss: 3.1486 - val_accuracy: 0.5095
Epoch 168/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.0796 - accuracy: 0.9729 - val_loss: 3.0988 - val_accuracy: 0.5000
Epoch 169/500
4728/4728 [==============================] - 0s 56us/sample - loss: 0.0878 - accuracy: 0.9721 - val_loss: 3.1874 - val_accuracy: 0.4886
Epoch 170/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.0916 - accuracy: 0.9685 - val_loss: 3.1797 - val_accuracy: 0.4886
Epoch 171/500
4728/4728 [==============================] - 0s 57us/sample - loss: 0.0841 - accuracy: 0.9695 - val_

KeyboardInterrupt: 